In [43]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [44]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application2_df=application_df.drop(columns=['EIN','NAME'])

In [45]:
#
app_type_counts=application2_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_app_type = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in replace_app_type:
    application2_df.APPLICATION_TYPE = application2_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application2_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [46]:
# Look at CLASSIFICATION value counts for binning
class_counts=application2_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application2_df.CLASSIFICATION = application2_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application2_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

# Base Case

In [47]:
# Retain application2_df as starting point for modifications
application3_df = application2_df


In [48]:
# Generate our categorical variable lists
var_cat = application3_df.dtypes[application3_df.dtypes=="object"].index.tolist()
var_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [49]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application3_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [50]:
# Merge one-hot encoded features and drop the originals
application4_df = application3_df.merge(encode_df, left_index=True, right_index=True)
application5_df = application4_df.drop(var_cat,1)
application5_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
# Split our preprocessed data into our features and target arrays
y = application5_df['IS_SUCCESSFUL'].values
X = application5_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

len(X_train[0])

43

In [53]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 80)                3520      
_________________________________________________________________
dense_25 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=30000)

In [55]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=30,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 3s 104us/sample - loss: 23475.8925 - accuracy: 0.4967
Epoch 2/30
 4192/25724 [===>..........................] - ETA: 1s - loss: 21664.6553 - accuracy: 0.4876
Epoch 00002: saving model to checkpoints/weights.02.hdf5
25724/25724 [==============================] - 2s 62us/sample - loss: 7035.1125 - accuracy: 0.4917
Epoch 3/30
 8544/25724 [========>.....................] - ETA: 1s - loss: 14582.9289 - accuracy: 0.5059
Epoch 00003: saving model to checkpoints/weights.03.hdf5
25724/25724 [==============================] - 2s 62us/sample - loss: 10978.4268 - accuracy: 0.4948
Epoch 4/30
12608/25724 [=============>................] - ETA: 0s - loss: 17363.2564 - accuracy: 0.5211
Epoch 00004: saving model to checkpoints/weights.04.hdf5
25724/25724 [==============================] - 2s 65us/sample - loss: 11072.2003 - accuracy: 0.5153
Epoch 5/30
16704/25724 [==================>...........] - ETA: 0s - loss: 2263.1933

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 1.6756 - accuracy: 0.5332
Loss: 1.675580564921521, Accuracy: 0.5331778526306152


# Begin Modification 1 - Delete STATUS

In [60]:
# Start with application2_df from above
application10_df = application2_df


In [61]:
# Delete STATUS
application10_df=application10_df.drop(columns=['STATUS'])

# Generate our categorical variable lists
var_cat = application10_df.dtypes[application10_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application10_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)

# Merge one-hot encoded features and drop the originals
application11_df = application10_df.merge(encode_df, left_index=True, right_index=True)
application12_df = application11_df.drop(var_cat,1)

# Split our preprocessed data into our features and target arrays
y1 = application12_df['IS_SUCCESSFUL'].values
X1 = application12_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X1_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn1.fit(X1_train,y1_train,epochs=30)


Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 2s 93us/sample - loss: 68777.0007 - accuracy: 0.4998
Epoch 2/30
25724/25724 [==============================] - 2s 60us/sample - loss: 65667.1889 - accuracy: 0.5062 0s - loss
Epoch 3/30
25724/25724 [==============================] - 2s 59us/sample - loss: 40053.9507 - accuracy: 0.4951
Epoch 4/30
25724/25724 [==============================] - 2s 59us/sample - loss: 45758.4196 - accuracy: 0.5142
Epoch 5/30
25724/25724 [==============================] - 2s 60us/sample - loss: 23264.7011 - accuracy: 0.5093
Epoch 6/30
25724/25724 [==============================] - 2s 60us/sample - loss: 8156.8081 - accuracy: 0.5175
Epoch 7/30
25724/25724 [==============================] - ETA: 0s - loss: 24090.9493 - accuracy: 0.50 - 2s 61us/sample - loss: 23563.7111 - accuracy: 0.5089
Epoch 8/30
25724/25724 [==============================] - 2s 64us/sample - loss: 3888.0774 - accuracy: 0.5157
Epoch 9/30
25724/25724 [===========

In [63]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6285 - accuracy: 0.6279
Loss: 0.6285409152820576, Accuracy: 0.6278716921806335


# Begin Modification 2 - Convert INCOME_AMT to object

In [64]:
# Start with application2_df from above
application20_df = application2_df

In [65]:
application20_df['INCOME_AMT']= application20_df['INCOME_AMT'].apply(str)
application20_df.dtypes

#income_counts=application2_df.INCOME_AMT.value_counts()
#income_counts

# Determine which values to replace
#replace_affiliation = list(affiliation_counts[affiliation_counts < 1000].index)

# Replace in dataframe
#for app in replace_affiliation:
#    application2_df.AFFILIATION = application2_df.AFFILIATION.replace(app,"Other")
    
# Check to make sure binning was successful
#application2_df.AFFILIATION.value_counts()

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [66]:
# Generate our categorical variable lists
var_cat = application20_df.dtypes[application20_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application20_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)

# Merge one-hot encoded features and drop the originals
application21_df = application20_df.merge(encode_df, left_index=True, right_index=True)
application22_df = application21_df.drop(var_cat,1)

# Split our preprocessed data into our features and target arrays
y2 = application22_df['IS_SUCCESSFUL'].values
X2 = application22_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler.fit(X2_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn2.fit(X_train,y_train,epochs=30)


Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 2s 82us/sample - loss: 60174.4028 - accuracy: 0.4993
Epoch 2/30
25724/25724 [==============================] - 2s 59us/sample - loss: 8579.2745 - accuracy: 0.4966
Epoch 3/30
25724/25724 [==============================] - 2s 65us/sample - loss: 4621.4061 - accuracy: 0.5058
Epoch 4/30
25724/25724 [==============================] - 1s 58us/sample - loss: 4740.9742 - accuracy: 0.4918
Epoch 5/30
25724/25724 [==============================] - 2s 58us/sample - loss: 0.7179 - accuracy: 0.5321
Epoch 6/30
25724/25724 [==============================] - 2s 61us/sample - loss: 0.6993 - accuracy: 0.5321
Epoch 7/30
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6927 - accuracy: 0.5321
Epoch 8/30
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6913 - accuracy: 0.5321
Epoch 9/30
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6911 - accuracy: 0.5321
E

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X2_test_scaled,y2_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6746 - accuracy: 0.5619
Loss: 0.674628225290393, Accuracy: 0.5618658661842346


# Begin Modification 3 - Modify AFFILIATION

In [69]:
# Start with application2_df from above
application30_df = application2_df

In [70]:
affiliation_counts=application30_df.AFFILIATION.value_counts()
affiliation_counts

# Determine which values to replace
replace_affiliation = list(affiliation_counts[affiliation_counts < 1000].index)

# Replace in dataframe
for app in replace_affiliation:
    application30_df.AFFILIATION = application30_df.AFFILIATION.replace(app,"Other")
    
# Check to make sure binning was successful
application30_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [71]:
# Generate our categorical variable lists
var_cat = application30_df.dtypes[application30_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application30_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)

# Merge one-hot encoded features and drop the originals
application31_df = application30_df.merge(encode_df, left_index=True, right_index=True)
application32_df = application31_df.drop(var_cat,1)

# Split our preprocessed data into our features and target arrays
y3 = application32_df['IS_SUCCESSFUL'].values
X3 = application32_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X3_scaler = scaler.fit(X3_train)

# Scale the data
X3_train_scaled = X3_scaler.transform(X3_train)
X3_test_scaled = X3_scaler.transform(X3_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X3_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn3.fit(X3_train,y3_train,epochs=30)


Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 2s 78us/sample - loss: 61940.6146 - accuracy: 0.5013
Epoch 2/30
25724/25724 [==============================] - 2s 59us/sample - loss: 13718.4671 - accuracy: 0.5022
Epoch 3/30
25724/25724 [==============================] - 2s 60us/sample - loss: 70348.5088 - accuracy: 0.4947
Epoch 4/30
25724/25724 [==============================] - 2s 59us/sample - loss: 55267.3909 - accuracy: 0.4910
Epoch 5/30
25724/25724 [==============================] - 1s 58us/sample - loss: 67922.5968 - accuracy: 0.5122
Epoch 6/30
25724/25724 [==============================] - 1s 58us/sample - loss: 31271.4603 - accuracy: 0.5214
Epoch 7/30
25724/25724 [==============================] - 2s 62us/sample - loss: 26250.9286 - accuracy: 0.5021
Epoch 8/30
25724/25724 [==============================] - 2s 60us/sample - loss: 9840.0845 - accuracy: 0.51690s - loss: 1773.7956 -
Epoch 9/30
25724/25724 [==============================] - 2s 64us/sa

In [72]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X3_test_scaled,y3_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.8148 - accuracy: 0.6007
Loss: 0.8147508702750804, Accuracy: 0.6006997227668762


# Begin Modification 4 - Increase neurons

In [73]:
# Start with application2_df from above
application40_df = application2_df

In [74]:
# Generate our categorical variable lists
var_cat = application40_df.dtypes[application40_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application40_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)

# Merge one-hot encoded features and drop the originals
application41_df = application40_df.merge(encode_df, left_index=True, right_index=True)
application42_df = application41_df.drop(var_cat,1)

# Split our preprocessed data into our features and target arrays
y4 = application42_df['IS_SUCCESSFUL'].values
X4 = application42_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X4_scaler = scaler.fit(X4_train)

# Scale the data
X4_train_scaled = X4_scaler.transform(X4_train)
X4_test_scaled = X4_scaler.transform(X4_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X4_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn4.fit(X4_train,y4_train,epochs=30)


Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 2s 92us/sample - loss: 62318.6833 - accuracy: 0.5062
Epoch 2/30
25724/25724 [==============================] - 2s 60us/sample - loss: 30020.6804 - accuracy: 0.4984
Epoch 3/30
25724/25724 [==============================] - 2s 62us/sample - loss: 26657.7750 - accuracy: 0.4912
Epoch 4/30
25724/25724 [==============================] - 2s 61us/sample - loss: 13979.1795 - accuracy: 0.5096
Epoch 5/30
25724/25724 [==============================] - 2s 62us/sample - loss: 9487.1851 - accuracy: 0.5070
Epoch 6/30
25724/25724 [==============================] - 2s 61us/sample - loss: 26027.3679 - accuracy: 0.5023
Epoch 7/30
25724/25724 [==============================] - 2s 62us/sample - loss: 32638.4750 - accuracy: 0.5074
Epoch 8/30
25724/25724 [==============================] - 2s 64us/sample - loss: 14122.5427 - accuracy: 0.5036
Epoch 9/30
25724/25724 [==============================] - 2s 76us/sample - loss: 5958.9922

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X4_test_scaled,y4_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 1.4587 - accuracy: 0.5664
Loss: 1.458723833387169, Accuracy: 0.5664139986038208


# Begin Modification 5 - Change Activation Functions

In [76]:
# Start with application2_df from above
application50_df = application2_df

In [77]:
# Generate our categorical variable lists
var_cat = application50_df.dtypes[application50_df.dtypes=="object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application50_df[var_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(var_cat)

# Merge one-hot encoded features and drop the originals
application51_df = application50_df.merge(encode_df, left_index=True, right_index=True)
application52_df = application51_df.drop(var_cat,1)

# Split our preprocessed data into our features and target arrays
y5 = application52_df['IS_SUCCESSFUL'].values
X5 = application52_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X5_scaler = scaler.fit(X5_train)

# Scale the data
X5_train_scaled = X5_scaler.transform(X5_train)
X5_test_scaled = X5_scaler.transform(X5_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X5_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn5 = tf.keras.models.Sequential()

# First hidden layer  ==> CHANGE TO sigmoid
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn5.fit(X5_train,y5_train,epochs=30)


Train on 25724 samples
Epoch 1/30
25724/25724 [==============================] - 2s 79us/sample - loss: 0.6937 - accuracy: 0.5196
Epoch 2/30
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6932 - accuracy: 0.5189
Epoch 3/30
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6926 - accuracy: 0.5224
Epoch 4/30
25724/25724 [==============================] - 2s 60us/sample - loss: 0.6924 - accuracy: 0.5230
Epoch 5/30
25724/25724 [==============================] - 2s 62us/sample - loss: 0.6921 - accuracy: 0.5267
Epoch 6/30
25724/25724 [==============================] - 2s 61us/sample - loss: 0.6924 - accuracy: 0.5252
Epoch 7/30
25724/25724 [==============================] - 2s 62us/sample - loss: 0.6918 - accuracy: 0.5287
Epoch 8/30
25724/25724 [==============================] - 2s 59us/sample - loss: 0.6917 - accuracy: 0.5321
Epoch 9/30
25724/25724 [==============================] - 2s 73us/sample - loss: 0.6916 - accuracy: 0.5298
Epoch 10/30
25

In [78]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X5_test_scaled,y5_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6865 - accuracy: 0.5311
Loss: 0.6864644073119316, Accuracy: 0.5310786962509155


# Save Model

In [80]:
# Save model
nn.save("AlphabetSoupCharity_Optimization.h5")